In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

data = pd.read_csv("rfm_data.csv")
print(data.head)

<bound method NDFrame.head of      CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0          8814     04/11/23             943.31          Product C   890075   
1          2188     04/11/23             463.70          Product A   176819   
2          4608     04/11/23              80.28          Product A   340062   
3          2559     04/11/23             221.29          Product A   239145   
4          9482     04/11/23             739.56          Product A   194545   
..          ...          ...                ...                ...      ...   
995        2970     06/10/23             759.62          Product B   275284   
996        6669     06/10/23             941.50          Product C   987025   
997        8836     06/10/23             545.36          Product C   512842   
998        1440     06/10/23             729.94          Product B   559753   
999        4759     06/10/23             804.28          Product D   467544   

     Location  
0    

In [2]:
from datetime import datetime

#converting purchasedate to datetime
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])

#calculating recency
data['Recency'] = (datetime.now().date() - data['PurchaseDate'].dt.date).dt.days

#calculating frequency
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)
data = data.merge(frequency_data, on='CustomerID', how='left')

#calculating monetary value
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)
data = data.merge(monetary_data, on='CustomerID', how='left')

print(data.head())


C:\Users\Darmian\AppData\Local\Temp\ipykernel_16580\3873355224.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])


AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
#defining scoring criteria
recency_scores = [5, 4, 3, 2, 1] #high score inferring more recent
monetary_scores = [1, 2, 3, 4, 5] #high score inferring higher frequency
frequency_scores = [1, 2, 3, 4, 5] #high score inferring higher monetary value

#calculating rfm scores
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)

In [ ]:
#converting rfm scores to intergers
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)

In [ ]:
# calculating rfm score by combining individual scores
data['RFM_Sco re'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# crreating rfm segments based on the rfm score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)